In [4]:
%run SAT_help_functions.ipynb

## OUM

In [5]:
data_dir = "/Users/wangjiashun/Desktop/Reserach on SAT/datasets/Evans2021/Evans2017/dataBySubject/sub"
sample_size = 47
SAT_conds = ["Speed", "Accuracy"]

In [35]:
model = Model(name='Simple model (fitted)',
                  drift=DriftConstant(drift = 1),                              
                  noise=NoiseConstant(noise=1),
                  bound=BoundConstant(B=1),
                  overlay=OverlayNonDecision(nondectime=0.1),
                  dx=.001, dt=.001, T_dur=3)
sol = model.solve()
samp = sol.resample(1000)
sample = samp.to_pandas_dataframe(drop_undecided = True)


In [34]:
#dir(samp)

In [74]:
samp = Sample.from_pandas_dataframe(df, rt_column_name="Time", correct_column_name="Correct")
get_DDM_result_sub_sv2a2ter(samp)

Info: Params [0.9009039  0.         0.55646615 0.85096549 0.3207267  0.39294593] gave 4232.924813675256


,Drift Rate,Drift Rate Variability,Boundary Separation Speed,Boundary Separation Accuracy,Non-decision Time Speed,Non-decision Time Accuracy,BIC
0,0.900904,0.0,0.556466,0.850965,0.320727,0.392946,4232.924814


In [83]:
df

,Drift Rate,Drift Rate Variability,Boundary Separation Speed,Boundary Separation Accuracy,Non-decision Time Speed,Non-decision Time Accuracy,BIC
0,1.292302,0.0,0.683041,1.400312,0.295892,0.504512,4258.540993


In [79]:

def prepare_sample_for_variable_drift(df, resolution=11):
    n = len(df)
    df = df.append([df]*10,ignore_index=True)
    DriftConditions = []
    for i in range(11):
        DriftConditions += [i]*n

    df['driftnum'] = DriftConditions
    return df

In [32]:
class DriftUniform(Drift):
    name = "Uniformly-distributed drift"
    resolution = RESOLUTION # Number of bins, should be an odd number
    required_parameters = ['drift', 'width'] # Mean drift and the width of the uniform distribution
    required_conditions = ['driftnum']
    def get_drift(self, conditions, **kwargs):
        stepsize = self.width/(self.resolution-1)
        mindrift = self.drift - self.width/2
        return mindrift + stepsize*conditions['driftnum']

In [8]:
class BoundSpeedAcc(Bound):
    name = "BoundSpeedAcc"
    required_parameters = ["Bspeed", "Bacc"]
    required_conditions = ['Cond']
    def get_bound(self, conditions, *args, **kwargs):
        assert self.Bacc > 0
        assert self.Bspeed > 0
        if conditions['Cond'] == 'Speed':
            return self.Bspeed
        elif conditions['Cond'] == 'Accuracy':
            return self.Bacc
        else:
            print("Non-implemented SAT conditions")
            
class NondecisionSpeedAcc(OverlayNonDecision):
    name = "NondecisionSpeedAcc"
    required_parameters = ["tSpeed", 'tAcc']
    required_conditions = ['Cond']
        
    def get_nondecision_time(self, conditions, **kwargs):
        if conditions['Cond'] == 'Speed':
            return self.tSpeed 
        elif conditions['Cond'] == 'Accuracy':
            return self.tAcc
        else: 
            print("Non-implemented SAT conditions")

In [81]:
def get_DDM_result_sub_sv2a2ter(samp, v = [0,6], sv = [0,1], a = [0.5,8], ter=[0.1,1], T_dur=3):
    
    model_fit = Model(name='Simple model (fitted)',
                      drift=DriftUniform(drift=Fittable(minval=v[0], maxval=v[1]),
                                         width=Fittable(minval=sv[0], maxval=sv[1])),                      
                      noise=NoiseConstant(noise=1),
                      bound=BoundSpeedAcc(
                          Bspeed=Fittable(minval=a[0], maxval=a[1]),
                          Bacc=Fittable(minval=a[0], maxval=a[1])),
                      
                      overlay=NondecisionSpeedAcc(
                          tSpeed=Fittable(minval=ter[0], maxval=ter[1]),
                          tAcc=Fittable(minval=ter[0], maxval=ter[1]),
                      ),
                      dx=.001, dt=.001, T_dur=T_dur)
    #set_N_cpus(4)
    fit_adjust_model(samp, model_fit,
                     fitting_method="differential_evolution",
                     lossfunction=LossRobustBIC, verbose=False)
    #print(model_fit.get_model_parameters())
    df = pd.DataFrame(data = np.array(model_fit.get_model_parameters()).reshape(1,6),
                      columns = model_fit.get_model_parameter_names())
    
    #df['Participant']=i+1
    #df['SAT'] = SAT
    df['BIC'] = model_fit.get_fit_result().value()
    df = df.rename(columns={'drift': 'Drift Rate',
                            'width':'Drift Rate Variability',
                            'Bspeed': 'Boundary Separation Speed',
                            'Bacc': 'Boundary Separation Accuracy',
                            'tSpeed': 'Non-decision Time Speed',
                            'tAcc': 'Non-decision Time Accuracy'})
    return df

In [82]:
df_all = pd.DataFrame()
for i in range(sample_size):
    print(i)
    path = os.path.join(data_dir+str(i+1)+".csv")
    df = pd.read_csv(path, sep = ",")
    #df = df[df["cond"]==SAT] # A condition
    df = df[df["Time"]<3]
    
    df = prepare_sample_for_variable_drift(df)

    samp = Sample.from_pandas_dataframe(df, rt_column_name="Time", correct_column_name="Correct")
    #samp = prepare_sample_for_variable_drift(samp, resolution=RESOLUTION)
    df = get_DDM_result_sub_sv2a2ter(samp)
    df_all = pd.concat((df_all,df))
    print(df)
df_OUM = df_all.copy()


0


Info: Params [0.89885075 0.         0.56127945 0.8496294  0.31787686 0.39058347] gave 4272.141603399549


   Drift Rate  Drift Rate Variability  Boundary Separation Speed  \
0    0.898851                     0.0                   0.561279   

   Boundary Separation Accuracy  Non-decision Time Speed  \
0                      0.849629                 0.317877   

   Non-decision Time Accuracy          BIC  
0                    0.390583  4272.141603  
1


Info: Params [1.88242207e+00 7.77526627e-07 5.64122881e-01 1.00736332e+00
 4.27322378e-01 5.19834405e-01] gave -919.7971977623714


   Drift Rate  Drift Rate Variability  Boundary Separation Speed  \
0    1.882422            7.775266e-07                   0.564123   

   Boundary Separation Accuracy  Non-decision Time Speed  \
0                      1.007363                 0.427322   

   Non-decision Time Accuracy         BIC  
0                    0.519834 -919.797198  
2


...

   Drift Rate  Drift Rate Variability  Boundary Separation Speed  \
0    1.536621                     0.0                   0.608481   

   Boundary Separation Accuracy  Non-decision Time Speed  \
0                      0.870032                 0.372375   

   Non-decision Time Accuracy          BIC  
0                    0.392286  1367.887993  
45


Info: Params [1.26021847 0.         0.6988195  0.82674454 0.35574919 0.42253053] gave 3538.317504612634


   Drift Rate  Drift Rate Variability  Boundary Separation Speed  \
0    1.260218                     0.0                    0.69882   

   Boundary Separation Accuracy  Non-decision Time Speed  \
0                      0.826745                 0.355749   

   Non-decision Time Accuracy          BIC  
0                    0.422531  3538.317505  
46


Info: Params [1.29230157 0.         0.68304127 1.40031155 0.29589224 0.50451246] gave 4258.540992972329


   Drift Rate  Drift Rate Variability  Boundary Separation Speed  \
0    1.292302                     0.0                   0.683041   

   Boundary Separation Accuracy  Non-decision Time Speed  \
0                      1.400312                 0.295892   

   Non-decision Time Accuracy          BIC  
0                    0.504512  4258.540993  


In [6]:
#df_OUM.to_csv("DDM2a2terresultEvans2017.csv")

In [7]:
#df_OUM["BIC"].mean()

358.4425613055732